# Little Loss sanity check for Hénon map values
* Here we just want to check if our weight and loss measurement system works properly

## Setup scripts if we are under SWAN

In [ ]:
# Working in the right path
%cd /eos/project/d/da-and-diffusion-studies/DA_Studies/Simulations/Models/loss_studies/notebooks

In [ ]:
# Install the libraries
import sys
!{sys.executable} -m pip install --user tqdm pynverse sixtrackwrap crank-nicolson-numba henon-map symplectic-map
!{sys.executable} -m pip install --user --upgrade sixtrackwrap 
!{sys.executable} -m pip install --user --upgrade crank-nicolson-numba 
!{sys.executable} -m pip install --user --upgrade henon-map 
!{sys.executable} -m pip install --user --upgrade symplectic-map
!export PYTHONPATH=$CERNBOX_HOME.local/lib/python3.7/site-packages:$PYTHONPATH

In [ ]:
# For this "presentation" only!
import warnings
warnings.filterwarnings('ignore')

## Library Imports

In [1]:
%matplotlib widget

In [2]:
# Base libraries
import math
import numpy as np
import scipy.integrate as integrate
from scipy.special import erf
import pickle
import itertools
from scipy.optimize import curve_fit

from numba import njit, prange

# Personal libraries
import sixtrackwrap as sx
import crank_nicolson_numba.nekhoroshev as nk
import henon_map as hm

from tqdm.notebook import tqdm
import time
import matplotlib.pyplot as plt
import ipywidgets as widgets
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.ticker as ticker
from math import gcd

import pandas as pd

from scipy.special import lambertw
from scipy.interpolate import interp1d

import warnings
import os

In [18]:
from fit_utils import symmetric_gaussian_loss

## Simple parameters

In [3]:
# We need a small value for this, otherwise the numerical discretization is macroscopic
dr = 0.001
ang_samples = 15
# This ends up with not needing that high value...
lin_samples = 30

max_turns = 1000
min_turns = 100

epsilon = 64.0

sample_list = np.linspace(min_turns, max_turns, 50)

In [4]:
# You can play with this one and try different values...
cutting_point = 0.9

In [5]:
alpha_preliminary_values = np.linspace(-1.0, 1.0, ang_samples)
alpha = np.arccos(alpha_preliminary_values) / 2
theta1 = np.linspace(0, np.pi * 2, ang_samples, endpoint=False)
theta2 = np.linspace(0, np.pi * 2, ang_samples, endpoint=False)

aa, th1, th2 = np.meshgrid(alpha, theta1, theta2, indexing="ij")

## Engines
### Radial one

In [6]:
radial_engine = hm.radial_scan.generate_instance(
    dr,
    aa.flatten(),
    th1.flatten(),
    th2.flatten(),
    epsilon,
    starting_position=0.3
)

In [7]:
radial_engine.block_compute(max_turns, min_turns);

In [8]:
radial_engine.save_values("../data/temp_radial.pkl")

### Uniform one

In [9]:
uniform_engine = hm.uniform_scan.generate_instance(
    epsilon,
    1.0,
    lin_samples,
    starting_radius = 0.3
)

In [10]:
uniform_engine.scan(max_turns);

Elapsed time for execution: 31.986249446868896 s


In [11]:
uniform_engine.save_values("../data/temp_uniform.pkl")

## Load values in the corresponding analyzers

In [12]:
radial_analyzer = sx.uniform_radial_scanner.load_values("../data/temp_radial.pkl")
uniform_analyzer = sx.uniform_scanner.load_values("../data/temp_uniform.pkl")

### Let's test a uniform distribution

In [ ]:
radial_analyzer.assign_weights(sx.assign_uniform_distribution())
uniform_analyzer.assign_weights(sx.assign_uniform_distribution(), radial_cut=cutting_point)

#### First sanity test - Manual raw comparison of boolean spheres 

In [ ]:
tests = np.linspace(0.1, 0.8, 200)
A = [] # theoretical value
B = [] # uniform sampling
C = [] # radial sampling
for test in tqdm(tests):
    A.append(np.pi**2 * test**4 / 2)
    B.append(uniform_analyzer.compute_loss_cut(test))
    C.append(radial_analyzer.compute_loss_cut(test))

In [ ]:
plt.figure()
plt.plot(tests, A, label="theoretical value")
#plt.plot(np.concatenate(([0,0,0],C)))
plt.plot(tests, B, label="uniform sampling")
plt.plot(tests, C, label="radial sampling")
plt.xlabel("4D sphere radius")
plt.ylabel("Measured hypervolume")
plt.legend()
plt.title("First sanity test")

#### Second sanity test - loss comparison (uniform distribution)

In [16]:
A = radial_analyzer.compute_loss(sample_list, cutting_point, normalization=False)
B = uniform_analyzer.compute_loss(sample_list, normalization=False)

In [17]:
plt.figure()
plt.plot(sample_list, A, label="Radial sampling")
plt.plot(sample_list, B, label="Uniform sampling")
plt.xlabel("Turns considered")
plt.ylabel("Active beam measured (not normalized)")
plt.legend()
plt.title("Second sanity test")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Second sanity test')

### Now on to a Gaussian distribution

In [74]:
sigma = 0.5

In [75]:
radial_analyzer.assign_weights(sx.assign_symmetric_gaussian(sigma))
uniform_analyzer.assign_weights(sx.assign_symmetric_gaussian(sigma, polar=False), radial_cut=cutting_point)

#### Third sanity test - loss comparison (gaussian distribution)

In [76]:
A = radial_analyzer.compute_loss(sample_list, cutting_point, normalization=True)
B = uniform_analyzer.compute_loss(sample_list, normalization=True)

In [77]:
DA = radial_analyzer.compute_DA_standard(sample_list)
C = symmetric_gaussian_loss(DA, sigma, cutting_point)

In [78]:
plt.figure()
plt.plot(sample_list, A, label="Radial sampling")
plt.plot(sample_list, B, label="Uniform sampling")
plt.plot(sample_list, C, label="DA loss")
plt.xlabel("Turns considered")
plt.ylabel("Active beam measured (not normalized)")
plt.legend()
plt.title("Third sanity test")

/home/carlidel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Third sanity test')

In [67]:
radiuses = radial_analyzer.extract_DA(sample_list)

In [84]:
plt.figure()
a = int(np.random.uniform(0, 15))
b = int(np.random.uniform(0, 15))
c = int(np.random.uniform(0, 15))
plt.plot(radial_analyzer.r_values, radial_analyzer.steps[a,b,c], linewidth=0.5, label="Raw steps measured\nin Hénon map")
plt.plot(radiuses[a,b,c], sample_list, c="C1", label="Last stable radius\nfor given number of turns")

plt.title("Random radius sample $(\\alpha={:.2},\\theta_1={:.2},\\theta_2={:.2})$".format(radial_analyzer.alpha_values[a], radial_analyzer.theta1_values[b], radial_analyzer.theta2_values[c]))
plt.xlabel("$r$")
plt.ylabel("$N$ turns")
plt.legend()

/home/carlidel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …